In [1]:
import pandas as pd 
import numpy as np

## Reading the data

In [2]:
vehicle_df = pd.read_excel('Vehcile_Data.xlsx')
vehicle_df.shape

(100, 5)

In [3]:
vehicle_charging_df = pd.read_excel('Vehcile_Charging_Data.xlsx')
vehicle_charging_df

,Year,TruckType,Range(Kms),VehicleBatteryCapacity(Kwh),FuelConsumption(Unit/km)
0,2025,1,400,360.0,0.90
1,2026,1,500,450.0,0.90
2,2027,1,550,495.0,0.90
3,2028,1,550,522.5,0.95
4,2029,1,570,541.5,0.95
5,2025,2,420,378.0,0.90
6,2026,2,520,468.0,0.90
7,2027,2,570,513.0,0.90
8,2028,2,580,551.0,0.95
9,2029,2,580,551.0,0.95


In [4]:
energy_profile_df = pd.read_excel('Energy_Profile_Data.xlsx')
energy_profile_df

,SlotsOfDay,BaseLoadConsumption(kWh),GridElectricityPrice (USD / kWh),SolarUnitsGeneration in Kwh/KwP
0,12AM-6 AM,7.1542,0.15,0.0
1,6AM-12 PM,3.9840,0.20,1.4
2,12PM-6PM,3.1175,0.25,1.8
3,6PM-12AM,7.0860,0.35,0.8


In [5]:
infra_procurement_df = pd.read_excel('Infra_Procurement_Costs.xlsx')
infra_procurement_df

,Year,Battery(USD/kWh),SolarPV(USD/kWp),Charger 50KW (USD/unit),Charger 400KW(USD/unit)
0,2025,300,450,1000,2500
1,2026,286,450,1000,2500
2,2027,272,450,1000,2500
3,2028,259,450,1000,2500
4,2029,247,450,1000,2500
5,2030,235,450,1000,2500
6,2031,224,360,1000,2500
7,2032,213,360,1000,2500
8,2033,203,360,1000,2500
9,2034,193,360,1000,2500


## Getting only infra procurement costs from 2025 to 2029¶

In [6]:
years_needed = [2025, 2026, 2027, 2028, 2029]
infra_costs_5yr = infra_procurement_df[infra_procurement_df["Year"].isin(years_needed)].copy()
infra_costs_5yr

,Year,Battery(USD/kWh),SolarPV(USD/kWp),Charger 50KW (USD/unit),Charger 400KW(USD/unit)
0,2025,300,450,1000,2500
1,2026,286,450,1000,2500
2,2027,272,450,1000,2500
3,2028,259,450,1000,2500
4,2029,247,450,1000,2500


## Merging vehicle data and charging data on year, trucktype to get fuel consumption (unit /km)

In [7]:
vehicle_with_fc = vehicle_df.merge(
    vehicle_charging_df[["Year", "TruckType", "FuelConsumption(Unit/km)"]],
    on=["Year", "TruckType"],
    how="left"
)
vehicle_with_fc

,VehicleID,TruckType,Year,DailyRange(Kms),ChargingSlot,FuelConsumption(Unit/km)
0,V001_2025,1,2025,250,6AM-12PM,0.90
1,V002_2025,1,2025,250,12PM-6PM,0.90
2,V003_2025,1,2025,250,6PM-12AM,0.90
3,V004_2025,1,2025,250,12AM-6AM,0.90
4,V005_2025,2,2025,280,6AM-12PM,0.90
...,...,...,...,...,...,...
95,V026_2029,1,2029,330,12PM-6PM,0.95
96,V027_2029,1,2029,330,6AM-12PM,0.95
97,V028_2029,1,2029,330,12PM-6PM,0.95
98,V029_2029,1,2029,330,6PM-12AM,0.95


## Getting Energy requirement per truck as range * fuel consumption

In [8]:
vehicle_with_fc['Energy_Requirement_Per_Truck'] = vehicle_with_fc['DailyRange(Kms)']*vehicle_with_fc['FuelConsumption(Unit/km)']
vehicle_with_fc

,VehicleID,TruckType,Year,DailyRange(Kms),ChargingSlot,FuelConsumption(Unit/km),Energy_Requirement_Per_Truck
0,V001_2025,1,2025,250,6AM-12PM,0.90,225.0
1,V002_2025,1,2025,250,12PM-6PM,0.90,225.0
2,V003_2025,1,2025,250,6PM-12AM,0.90,225.0
3,V004_2025,1,2025,250,12AM-6AM,0.90,225.0
4,V005_2025,2,2025,280,6AM-12PM,0.90,252.0
...,...,...,...,...,...,...,...
95,V026_2029,1,2029,330,12PM-6PM,0.95,313.5
96,V027_2029,1,2029,330,6AM-12PM,0.95,313.5
97,V028_2029,1,2029,330,12PM-6PM,0.95,313.5
98,V029_2029,1,2029,330,6PM-12AM,0.95,313.5


## Creating a slot map to make sure time slots can be listed as 1,2,3,4 for easier use later

In [9]:
slot_map = {
    "12AM-6AM": 1,
    "6AM-12PM": 2,
    "12PM-6PM": 3,
    "6PM-12AM": 4
}

vehicle_with_fc["SlotID"] = vehicle_with_fc["ChargingSlot"].map(slot_map)
vehicle_with_fc.head()

,VehicleID,TruckType,Year,DailyRange(Kms),ChargingSlot,FuelConsumption(Unit/km),Energy_Requirement_Per_Truck,SlotID
0,V001_2025,1,2025,250,6AM-12PM,0.9,225.0,2
1,V002_2025,1,2025,250,12PM-6PM,0.9,225.0,3
2,V003_2025,1,2025,250,6PM-12AM,0.9,225.0,4
3,V004_2025,1,2025,250,12AM-6AM,0.9,225.0,1
4,V005_2025,2,2025,280,6AM-12PM,0.9,252.0,2


In [10]:
slot_map_energy = {
    "12AM-6 AM": 1,
    "6AM-12 PM": 2,
    "12PM-6PM": 3,
    "6PM-12AM": 4
}

energy_profile_df["SlotID"] = energy_profile_df["SlotsOfDay"].map(slot_map_energy)
energy_profile_df.head()

,SlotsOfDay,BaseLoadConsumption(kWh),GridElectricityPrice (USD / kWh),SolarUnitsGeneration in Kwh/KwP,SlotID
0,12AM-6 AM,7.1542,0.15,0.0,1
1,6AM-12 PM,3.9840,0.20,1.4,2
2,12PM-6PM,3.1175,0.25,1.8,3
3,6PM-12AM,7.0860,0.35,0.8,4


## Merging with energy data to ensure grid price, solar generation and baseload cosnumption are present with vehicle data and vehicle charging data

In [11]:
vehicle_with_energy = vehicle_with_fc.merge(
    energy_profile_df[["SlotID", "BaseLoadConsumption(kWh)", 
                       "GridElectricityPrice (USD / kWh)",
                       "SolarUnitsGeneration in Kwh/KwP"]],
    on="SlotID",
    how="left"
)
vehicle_with_energy

,VehicleID,TruckType,Year,DailyRange(Kms),ChargingSlot,FuelConsumption(Unit/km),Energy_Requirement_Per_Truck,SlotID,BaseLoadConsumption(kWh),GridElectricityPrice (USD / kWh),SolarUnitsGeneration in Kwh/KwP
0,V001_2025,1,2025,250,6AM-12PM,0.90,225.0,2,3.9840,0.20,1.4
1,V002_2025,1,2025,250,12PM-6PM,0.90,225.0,3,3.1175,0.25,1.8
2,V003_2025,1,2025,250,6PM-12AM,0.90,225.0,4,7.0860,0.35,0.8
3,V004_2025,1,2025,250,12AM-6AM,0.90,225.0,1,7.1542,0.15,0.0
4,V005_2025,2,2025,280,6AM-12PM,0.90,252.0,2,3.9840,0.20,1.4
...,...,...,...,...,...,...,...,...,...,...,...
95,V026_2029,1,2029,330,12PM-6PM,0.95,313.5,3,3.1175,0.25,1.8
96,V027_2029,1,2029,330,6AM-12PM,0.95,313.5,2,3.9840,0.20,1.4
97,V028_2029,1,2029,330,12PM-6PM,0.95,313.5,3,3.1175,0.25,1.8
98,V029_2029,1,2029,330,6PM-12AM,0.95,313.5,4,7.0860,0.35,0.8


## Getting the requirements per slot by using group by on year and slot Id, sums up all trucks using that slot for charging¶

In [12]:
slot_charging_req = (
    vehicle_with_energy.groupby(["Year", "SlotID"])["Energy_Requirement_Per_Truck"]
    .sum()
    .reset_index()
    .rename(columns={"Energy_Requirement_Per_Truck": "ChargingRequirement"})
)
slot_charging_req

,Year,SlotID,ChargingRequirement
0,2025,1,477.0
1,2025,2,702.0
2,2025,3,729.0
3,2025,4,477.0
4,2026,1,477.0
5,2026,2,1197.0
6,2026,3,1233.0
7,2026,4,720.0
8,2027,1,1008.0
9,2027,2,1764.0


## Merging slot data with energy profile for the same reason as before, to get base load, grid price and solar units.¶

In [13]:
slot_data = slot_charging_req.merge(
    energy_profile_df[["SlotID",
                       "BaseLoadConsumption(kWh)",
                       "GridElectricityPrice (USD / kWh)",
                       "SolarUnitsGeneration in Kwh/KwP"]],
    on="SlotID",
    how="left"
)
slot_data

,Year,SlotID,ChargingRequirement,BaseLoadConsumption(kWh),GridElectricityPrice (USD / kWh),SolarUnitsGeneration in Kwh/KwP
0,2025,1,477.0,7.1542,0.15,0.0
1,2025,2,702.0,3.9840,0.20,1.4
2,2025,3,729.0,3.1175,0.25,1.8
3,2025,4,477.0,7.0860,0.35,0.8
4,2026,1,477.0,7.1542,0.15,0.0
5,2026,2,1197.0,3.9840,0.20,1.4
6,2026,3,1233.0,3.1175,0.25,1.8
7,2026,4,720.0,7.0860,0.35,0.8
8,2027,1,1008.0,7.1542,0.15,0.0
9,2027,2,1764.0,3.9840,0.20,1.4


## Total requirement in a slot = Charing Requirement + Base Load

In [14]:
slot_data["TotalEnergyRequirement"] = (
    slot_data["ChargingRequirement"] + slot_data["BaseLoadConsumption(kWh)"]
)

In [15]:
slot_data

,Year,SlotID,ChargingRequirement,BaseLoadConsumption(kWh),GridElectricityPrice (USD / kWh),SolarUnitsGeneration in Kwh/KwP,TotalEnergyRequirement
0,2025,1,477.0,7.1542,0.15,0.0,484.1542
1,2025,2,702.0,3.9840,0.20,1.4,705.9840
2,2025,3,729.0,3.1175,0.25,1.8,732.1175
3,2025,4,477.0,7.0860,0.35,0.8,484.0860
4,2026,1,477.0,7.1542,0.15,0.0,484.1542
5,2026,2,1197.0,3.9840,0.20,1.4,1200.9840
6,2026,3,1233.0,3.1175,0.25,1.8,1236.1175
7,2026,4,720.0,7.0860,0.35,0.8,727.0860
8,2027,1,1008.0,7.1542,0.15,0.0,1015.1542
9,2027,2,1764.0,3.9840,0.20,1.4,1767.9840


## Some fixed parameters

In [16]:
battery_max_buy_overall = 2000
C_rate = 0.5
grid_upper_limit_per_slot = 1750
present_solar = 500
available_solar_to_buy_overall = 200
capex_budget_per_year = 60000

## Infrastructure costs

In [17]:
infra_costs_5yr

,Year,Battery(USD/kWh),SolarPV(USD/kWp),Charger 50KW (USD/unit),Charger 400KW(USD/unit)
0,2025,300,450,1000,2500
1,2026,286,450,1000,2500
2,2027,272,450,1000,2500
3,2028,259,450,1000,2500
4,2029,247,450,1000,2500


In [18]:
# Create dictionaries for cost inputs
battery_cost = dict(zip(infra_costs_5yr["Year"], infra_costs_5yr["Battery(USD/kWh)"]))
solar_cost = dict(zip(infra_costs_5yr["Year"], infra_costs_5yr["SolarPV(USD/kWp)"]))
charger50_cost = dict(zip(infra_costs_5yr["Year"], infra_costs_5yr["Charger 50KW (USD/unit)"]))
charger400_cost = dict(zip(infra_costs_5yr["Year"], infra_costs_5yr["Charger 400KW(USD/unit)"]))

battery_cost, solar_cost, charger50_cost, charger400_cost


({2025: 300, 2026: 286, 2027: 272, 2028: 259, 2029: 247},
 {2025: 450, 2026: 450, 2027: 450, 2028: 450, 2029: 450},
 {2025: 1000, 2026: 1000, 2027: 1000, 2028: 1000, 2029: 1000},
 {2025: 2500, 2026: 2500, 2027: 2500, 2028: 2500, 2029: 2500})

## Optimization

## Defining various lists and dictionaries related to slots for ease of use in constraints¶

In [19]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model('5 Year CAPEX and OPEX Planning')

## Defining lists and dictionaries ##

years = sorted(slot_data["Year"].unique())
slots = sorted(slot_data["SlotID"].unique())

slot_charging_requirement = {
    (row.Year, row.SlotID): row.ChargingRequirement
    for _, row in slot_data.iterrows()
}

slot_baseload = {
    (row.Year, row.SlotID): row["BaseLoadConsumption(kWh)"]
    for _, row in slot_data.iterrows()
}

slot_grid_price = {
    (row.Year, row.SlotID): row["GridElectricityPrice (USD / kWh)"]
    for _, row in slot_data.iterrows()
}

slot_solar_factor = {
    (row.Year, row.SlotID): row["SolarUnitsGeneration in Kwh/KwP"]
    for _, row in slot_data.iterrows()
}

slot_total_demand = {
    (row.Year, row.SlotID): row.TotalEnergyRequirement
    for _, row in slot_data.iterrows()
}


Set parameter Username
Set parameter LicenseID to value 2674748
Academic license - for non-commercial use only - expires 2026-06-03


In [20]:
slot_charging_requirement

{(2025.0, 1.0): 477.0,
 (2025.0, 2.0): 702.0,
 (2025.0, 3.0): 729.0,
 (2025.0, 4.0): 477.0,
 (2026.0, 1.0): 477.0,
 (2026.0, 2.0): 1197.0,
 (2026.0, 3.0): 1233.0,
 (2026.0, 4.0): 720.0,
 (2027.0, 1.0): 1008.0,
 (2027.0, 2.0): 1764.0,
 (2027.0, 3.0): 1260.0,
 (2027.0, 4.0): 1008.0,
 (2028.0, 1.0): 1121.0,
 (2028.0, 2.0): 2213.5,
 (2028.0, 3.0): 2147.0,
 (2028.0, 4.0): 1358.5,
 (2029.0, 1.0): 1102.0,
 (2029.0, 2.0): 2508.0,
 (2029.0, 3.0): 3097.0,
 (2029.0, 4.0): 1653.0}

In [21]:
slot_baseload

{(2025.0, 1.0): 7.1542,
 (2025.0, 2.0): 3.984,
 (2025.0, 3.0): 3.1175,
 (2025.0, 4.0): 7.086,
 (2026.0, 1.0): 7.1542,
 (2026.0, 2.0): 3.984,
 (2026.0, 3.0): 3.1175,
 (2026.0, 4.0): 7.086,
 (2027.0, 1.0): 7.1542,
 (2027.0, 2.0): 3.984,
 (2027.0, 3.0): 3.1175,
 (2027.0, 4.0): 7.086,
 (2028.0, 1.0): 7.1542,
 (2028.0, 2.0): 3.984,
 (2028.0, 3.0): 3.1175,
 (2028.0, 4.0): 7.086,
 (2029.0, 1.0): 7.1542,
 (2029.0, 2.0): 3.984,
 (2029.0, 3.0): 3.1175,
 (2029.0, 4.0): 7.086}

In [22]:
slot_grid_price

{(2025.0, 1.0): 0.15,
 (2025.0, 2.0): 0.2,
 (2025.0, 3.0): 0.25,
 (2025.0, 4.0): 0.35,
 (2026.0, 1.0): 0.15,
 (2026.0, 2.0): 0.2,
 (2026.0, 3.0): 0.25,
 (2026.0, 4.0): 0.35,
 (2027.0, 1.0): 0.15,
 (2027.0, 2.0): 0.2,
 (2027.0, 3.0): 0.25,
 (2027.0, 4.0): 0.35,
 (2028.0, 1.0): 0.15,
 (2028.0, 2.0): 0.2,
 (2028.0, 3.0): 0.25,
 (2028.0, 4.0): 0.35,
 (2029.0, 1.0): 0.15,
 (2029.0, 2.0): 0.2,
 (2029.0, 3.0): 0.25,
 (2029.0, 4.0): 0.35}

In [23]:
slot_solar_factor

{(2025.0, 1.0): 0.0,
 (2025.0, 2.0): 1.4,
 (2025.0, 3.0): 1.8,
 (2025.0, 4.0): 0.8,
 (2026.0, 1.0): 0.0,
 (2026.0, 2.0): 1.4,
 (2026.0, 3.0): 1.8,
 (2026.0, 4.0): 0.8,
 (2027.0, 1.0): 0.0,
 (2027.0, 2.0): 1.4,
 (2027.0, 3.0): 1.8,
 (2027.0, 4.0): 0.8,
 (2028.0, 1.0): 0.0,
 (2028.0, 2.0): 1.4,
 (2028.0, 3.0): 1.8,
 (2028.0, 4.0): 0.8,
 (2029.0, 1.0): 0.0,
 (2029.0, 2.0): 1.4,
 (2029.0, 3.0): 1.8,
 (2029.0, 4.0): 0.8}

In [24]:
slot_total_demand

{(2025.0, 1.0): 484.1542,
 (2025.0, 2.0): 705.984,
 (2025.0, 3.0): 732.1175,
 (2025.0, 4.0): 484.086,
 (2026.0, 1.0): 484.1542,
 (2026.0, 2.0): 1200.984,
 (2026.0, 3.0): 1236.1175,
 (2026.0, 4.0): 727.086,
 (2027.0, 1.0): 1015.1542,
 (2027.0, 2.0): 1767.984,
 (2027.0, 3.0): 1263.1175,
 (2027.0, 4.0): 1015.086,
 (2028.0, 1.0): 1128.1542,
 (2028.0, 2.0): 2217.484,
 (2028.0, 3.0): 2150.1175,
 (2028.0, 4.0): 1365.586,
 (2029.0, 1.0): 1109.1542,
 (2029.0, 2.0): 2511.984,
 (2029.0, 3.0): 3100.1175,
 (2029.0, 4.0): 1660.086}

## Decision Variables ##

## These have been formulated in the same way as the report, use of for loops in certain places for convenience.¶

In [25]:
solar_buy_year = model.addVars(years,vtype = GRB.CONTINUOUS,lb = 0,name = 'Amount of solar to buy each year')
solar_used_year_slot = model.addVars(years,slots,vtype = GRB.CONTINUOUS,lb = 0,name = 'Amount of solar energy used in each slot each year')
solar_export_year_slot = model.addVars(years,slots,vtype = GRB.CONTINUOUS,lb = 0,name = 'Amount of solar energy exported in each slot each year')
solar_stored_battery_slot = model.addVars(years,slots,vtype = GRB.CONTINUOUS,lb = 0,name = 'Amount of solar energy stored in batteries in each slot each year')

battery_buy_year = model.addVars(years,vtype = GRB.CONTINUOUS,lb = 0,name = 'Amount of battery capacity to buy each year')
battery_used_year_slot = model.addVars(years,slots,vtype = GRB.CONTINUOUS,lb = 0,name = 'Amount of energy used from batteries in each slot each year')
battery_export_year_slot =  model.addVars(years,slots,vtype = GRB.CONTINUOUS,lb = 0,name = 'Amount of energy exported from batteries to grid in each slot each year')
battery_start_year_slot = model.addVars(years,slots,vtype = GRB.CONTINUOUS,lb = 0,name = 'Energy in battery at start of each slot each year')

grid_year_slot = model.addVars(years,slots,vtype = GRB.CONTINUOUS,lb = 0,name = 'Amount of energy used from the grid in slot each slot each year')

nocfifty = model.addVars(years,vtype = GRB.INTEGER,lb = 0,name = 'Number of 50 kW Chargers to buy each year')
nocfourhundred = model.addVars(years,vtype = GRB.INTEGER,lb = 0,name = 'Number of 400 kW Chargers to buy each year')

battery_capacity_per_year = model.addVars(years,vtype = GRB.CONTINUOUS,lb = 0)
for idx, y in enumerate(years):
    if idx == 0:   # first year (2025)
        model.addConstr(
            battery_capacity_per_year[y] == battery_buy_year[y]
        )
    else:
        prev = years[idx - 1]
        model.addConstr(
            battery_capacity_per_year[y] ==
                battery_capacity_per_year[prev] + battery_buy_year[y]
        )
        
nocfifty_cap_per_year = model.addVars(years,vtype = GRB.INTEGER,lb = 0)
nocfourhundred_cap_per_year = model.addVars(years,vtype = GRB.INTEGER,lb = 0)

for idx,y in enumerate(years):
    if idx == 0:
        model.addConstr(nocfifty_cap_per_year[y] == nocfifty[y])
        model.addConstr(nocfourhundred_cap_per_year[y] == nocfourhundred[y])
    else:
        prev = years[idx - 1]
        model.addConstr(nocfifty_cap_per_year[y] == nocfifty_cap_per_year[prev] + nocfifty[y])
        model.addConstr(nocfourhundred_cap_per_year[y] == nocfourhundred_cap_per_year[prev] + nocfourhundred[y])

## Solar generation as a constraint, same as report formulation

In [26]:
solar_gen_year_slot = model.addVars(years,slots,vtype = GRB.CONTINUOUS,lb = 0)

for y in years:
    for s in slots:
        model.addConstr(solar_gen_year_slot[y,s] == (present_solar +(gp.quicksum(solar_buy_year[k] for k in years if k<=y)))*slot_solar_factor[(y,s)])

In [27]:
slot_solar_factor

{(2025.0, 1.0): 0.0,
 (2025.0, 2.0): 1.4,
 (2025.0, 3.0): 1.8,
 (2025.0, 4.0): 0.8,
 (2026.0, 1.0): 0.0,
 (2026.0, 2.0): 1.4,
 (2026.0, 3.0): 1.8,
 (2026.0, 4.0): 0.8,
 (2027.0, 1.0): 0.0,
 (2027.0, 2.0): 1.4,
 (2027.0, 3.0): 1.8,
 (2027.0, 4.0): 0.8,
 (2028.0, 1.0): 0.0,
 (2028.0, 2.0): 1.4,
 (2028.0, 3.0): 1.8,
 (2028.0, 4.0): 0.8,
 (2029.0, 1.0): 0.0,
 (2029.0, 2.0): 1.4,
 (2029.0, 3.0): 1.8,
 (2029.0, 4.0): 0.8}

## Constraints (Same as report formulation)

In [28]:
model.addConstr(gp.quicksum(solar_buy_year[y] for y in years) <= available_solar_to_buy_overall)
model.addConstr(gp.quicksum(battery_buy_year[y] for y in years) <= battery_max_buy_overall)

for y in years:
    for s in slots:
        model.addConstr(solar_used_year_slot[y,s] + grid_year_slot[y,s] + battery_used_year_slot[y,s] == slot_total_demand[y,s])
        model.addConstr(solar_export_year_slot[y,s] + solar_stored_battery_slot[y,s] + solar_used_year_slot[y,s] == solar_gen_year_slot[y,s])
       
        #if y == 2025:
            #model.addConstr(battery_start_year_slot[2025,1] == 0)   
        #else:
            #model.addConstr(battery_start_year_slot[y,1] == battery_start_year_slot[y-1,4] + solar_stored_battery_slot[y-1,4] - 
                           # battery_used_year_slot[y-1,4] - battery_export_year_slot[y-1,4])         
        if s == 1:
            model.addConstr(battery_start_year_slot[y,s] == battery_start_year_slot[y,4] + solar_stored_battery_slot[y,4] - 
                            battery_used_year_slot[y,4] - battery_export_year_slot[y,4])
        else:
            model.addConstr(battery_start_year_slot[y,s] == battery_start_year_slot[y,s-1] + solar_stored_battery_slot[y,s-1] -
                           battery_used_year_slot[y,s-1] - battery_export_year_slot[y,s-1])
        model.addConstr(battery_used_year_slot[y,s] + battery_export_year_slot[y,s] <= C_rate*battery_capacity_per_year[y])
        model.addConstr(battery_start_year_slot[y,s] <= battery_capacity_per_year[y])
        model.addConstr((nocfifty_cap_per_year[y] * 50 + nocfourhundred_cap_per_year[y] * 400) * 6 >= slot_charging_requirement[y,s])
        model.addConstr(grid_year_slot[y,s] <= grid_upper_limit_per_slot)



In [29]:
slot_charging_requirement

{(2025.0, 1.0): 477.0,
 (2025.0, 2.0): 702.0,
 (2025.0, 3.0): 729.0,
 (2025.0, 4.0): 477.0,
 (2026.0, 1.0): 477.0,
 (2026.0, 2.0): 1197.0,
 (2026.0, 3.0): 1233.0,
 (2026.0, 4.0): 720.0,
 (2027.0, 1.0): 1008.0,
 (2027.0, 2.0): 1764.0,
 (2027.0, 3.0): 1260.0,
 (2027.0, 4.0): 1008.0,
 (2028.0, 1.0): 1121.0,
 (2028.0, 2.0): 2213.5,
 (2028.0, 3.0): 2147.0,
 (2028.0, 4.0): 1358.5,
 (2029.0, 1.0): 1102.0,
 (2029.0, 2.0): 2508.0,
 (2029.0, 3.0): 3097.0,
 (2029.0, 4.0): 1653.0}

## Minimize capex + opex - revenue (solar + battery).Capex consists of new stuff you buy and opex is buying from the grid.

In [30]:
battery_cost      = infra_costs_5yr.set_index("Year")["Battery(USD/kWh)"].to_dict()
solar_cost        = infra_costs_5yr.set_index("Year")["SolarPV(USD/kWp)"].to_dict()
charger50_cost    = infra_costs_5yr.set_index("Year")["Charger 50KW (USD/unit)"].to_dict()
charger400_cost   = infra_costs_5yr.set_index("Year")["Charger 400KW(USD/unit)"].to_dict()

capex_year = {}
opex_year = {}
#solar_export_rev_year = {}
#battery_export_rev_year = {}

for y in years:

    # CAPEX
    capex_year[y] = (
        solar_cost[y]   * solar_buy_year[y] +
        battery_cost[y] * battery_buy_year[y] +
        charger50_cost[y]  * nocfifty[y] +
        charger400_cost[y] * nocfourhundred[y]
    )

    model.addConstr(capex_year[y] <= 60000)

    # OPEX
    opex_year[y] = 365 * gp.quicksum(
        grid_year_slot[y,s] * slot_grid_price[(y,s)] for s in slots
    )

    # Revenues
    #solar_export_rev_year[y] = 365 * gp.quicksum(
    #solar_export_year_slot[y,s] * slot_grid_price[(y,s)] for s in slots
    #)

    #battery_export_rev_year[y] = 365 * gp.quicksum(
     #   battery_export_year_slot[y,s] * slot_grid_price[(y,s)] for s in slots
    #)

# Total objective over 5 years
model.setObjective(
    gp.quicksum(capex_year[y] for y in years) +
    gp.quicksum(opex_year[y] for y in years),GRB.MINIMIZE)
    #gp.quicksum(solar_export_rev_year[y] for y in years) -
    #gp.quicksum(battery_export_rev_year[y] for y in years),GRB.MINIMIZE)
    



In [31]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 182 rows, 195 columns and 537 nonzeros
Model fingerprint: 0x07c3a834
Variable types: 175 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+03]
  Objective range  [5e+01, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+04]
Presolve removed 89 rows and 99 columns
Presolve time: 0.00s
Presolved: 93 rows, 96 columns, 298 nonzeros
Variable types: 84 continuous, 12 integer (0 binary)
Found heuristic solution: objective 1356434.6232

Root relaxation: objective 1.199381e+06, 59 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

## Display of results year by year slot by slot. Used AI for the coding (attached in appendix of report)

In [32]:
if model.status == GRB.OPTIMAL:
    print("\n==========================")
    print(" TOTAL COST OVER 5 YEARS ")
    print("==========================")
    print(f"Objective Value: {model.ObjVal:,.2f}\n")

    # ========= YEARLY REPORTS ========= #
    for y in years:
        print("\n====================================")
        print(f" YEAR {y} RESULTS")
        print("====================================")

        # CAPEX Categories
        print("\n---- CAPEX ----")
        print(f"Solar Bought (kWp): {solar_buy_year[y].X:,.2f}")
        print(f"Battery Bought (kWh): {battery_buy_year[y].X:,.2f}")
        print(f"50 kW Chargers Bought: {nocfifty[y].X:,.0f}")
        print(f"400 kW Chargers Bought: {nocfourhundred[y].X:,.0f}")
        print(f"Total CAPEX (USD): {capex_year[y].getValue():,.2f}")

        # OPEX
        print("\n---- OPEX ----")
        print(f"Grid OPEX per Year (USD): {opex_year[y].getValue():,.2f}")

        # Revenues
        print("\n---- EXPORT REVENUES ----")
       # print(f"Solar Export Revenue (USD): {solar_export_rev_year[y].getValue():,.2f}")
       # print(f"Battery Export Revenue (USD): {battery_export_rev_year[y].getValue():,.2f}")

        # ========= SLOT DETAILS ========= #
        print("\n---- SLOT DETAILS ----")
        for s in slots:
            print(f"\n Slot {s}:")

            # Grid energy
            print(f"  Grid Energy Used (kWh): {grid_year_slot[y,s].X:.2f}")

            # Solar
            print("  Solar:")
            print(f"    Generated: {solar_gen_year_slot[y,s].X:.2f}")
            print(f"    Used:      {solar_used_year_slot[y,s].X:.2f}")
            print(f"    Stored:    {solar_stored_battery_slot[y,s].X:.2f}")
            print(f"    Exported:  {solar_export_year_slot[y,s].X:.2f}")

            # Battery
            print("  Battery:")
            print(f"    SOC Start:  {battery_start_year_slot[y,s].X:.2f}")
            print(f"    Used:       {battery_used_year_slot[y,s].X:.2f}")
            print(f"    Exported:   {battery_export_year_slot[y,s].X:.2f}")

            # Energy balance check
            lhs = (
                solar_used_year_slot[y,s].X +
                grid_year_slot[y,s].X +
                battery_used_year_slot[y,s].X
            )
            rhs = slot_total_demand[(y,s)]
            diff = lhs - rhs

            print("\n  Energy Balance Check:")
            print(f"    LHS = {lhs:.3f}, RHS = {rhs:.3f}, Difference = {diff:.6f}")

    print("\nAll reports generated successfully.")

else:
    print("Model not optimal. Status:", model.status)



 TOTAL COST OVER 5 YEARS 
Objective Value: 1,200,943.96


 YEAR 2025 RESULTS

---- CAPEX ----
Solar Bought (kWp): 66.67
Battery Bought (kWh): 91.67
50 kW Chargers Bought: 0
400 kW Chargers Bought: 1
Total CAPEX (USD): 60,000.00

---- OPEX ----
Grid OPEX per Year (USD): 23,998.07

---- EXPORT REVENUES ----

---- SLOT DETAILS ----

 Slot 1:
  Grid Energy Used (kWh): 438.32
  Solar:
    Generated: 0.00
    Used:      0.00
    Stored:    0.00
    Exported:  0.00
  Battery:
    SOC Start:  45.83
    Used:       45.83
    Exported:   0.00

  Energy Balance Check:
    LHS = 484.154, RHS = 484.154, Difference = 0.000000

 Slot 2:
  Grid Energy Used (kWh): 0.00
  Solar:
    Generated: 793.33
    Used:      705.98
    Stored:    0.00
    Exported:  87.35
  Battery:
    SOC Start:  0.00
    Used:       0.00
    Exported:   0.00

  Energy Balance Check:
    LHS = 705.984, RHS = 705.984, Difference = 0.000000

 Slot 3:
  Grid Energy Used (kWh): 0.00
  Solar:
    Generated: 1020.00
    Used:      7

In [33]:
if model.status == GRB.OPTIMAL:
    print("\n==========================")
    print(" TOTAL COST OVER 5 YEARS ")
    print("==========================")
    print(f"Objective Value: {model.ObjVal:,.2f}\n")

    # ========= YEARLY REPORTS ========= #
    for y in years:
        print("\n====================================")
        print(f" YEAR {y} RESULTS")
        print("====================================")

        # CAPEX Categories
        print("\n---- CAPEX ----")
        print(f"Solar Bought This Year (kWp): {solar_buy_year[y].X:,.2f}")
        print(f"Battery Bought This Year (kWh): {battery_buy_year[y].X:,.2f}")
        print(f"50 kW Chargers Bought This Year: {nocfifty[y].X:,.0f}")
        print(f"400 kW Chargers Bought This Year: {nocfourhundred[y].X:,.0f}")
        print(f"Total CAPEX (USD): {capex_year[y].getValue():,.2f}")

        # CAPACITY AVAILABLE AFTER PURCHASES
        print("\n---- INSTALLED CAPACITY (CUMULATIVE) ----")
        print(f"Total Battery Capacity Available (kWh): {battery_capacity_per_year[y].X:,.2f}")
        print(f"Total 50 kW Chargers Available: {nocfifty_cap_per_year[y].X:,.0f}")
        print(f"Total 400 kW Chargers Available: {nocfourhundred_cap_per_year[y].X:,.0f}")

        # OPEX
        print("\n---- OPEX ----")
        print(f"Grid OPEX per Year (USD): {opex_year[y].getValue():,.2f}")

        # Revenues
        print("\n---- EXPORT REVENUES ----")
        #print(f"Solar Export Revenue (USD): {solar_export_rev_year[y].getValue():,.2f}")
        #print(f"Battery Export Revenue (USD): {battery_export_rev_year[y].getValue():,.2f}")

        # ========= SLOT DETAILS ========= #
        print("\n---- SLOT DETAILS ----")
        for s in slots:
            print(f"\n Slot {s}:")

            # Grid energy
            print(f"  Grid Energy Used (kWh): {grid_year_slot[y,s].X:.2f}")

            # Solar
            print("  Solar:")
            print(f"    Generated: {solar_gen_year_slot[y,s].X:.2f}")
            print(f"    Used:      {solar_used_year_slot[y,s].X:.2f}")
            print(f"    Stored:    {solar_stored_battery_slot[y,s].X:.2f}")
            print(f"    Exported:  {solar_export_year_slot[y,s].X:.2f}")

            # Battery
            print("  Battery:")
            print(f"    SOC Start:  {battery_start_year_slot[y,s].X:.2f}")
            print(f"    Used:       {battery_used_year_slot[y,s].X:.2f}")
            print(f"    Exported:   {battery_export_year_slot[y,s].X:.2f}")

            # Energy balance check
            lhs = (
                solar_used_year_slot[y,s].X +
                grid_year_slot[y,s].X +
                battery_used_year_slot[y,s].X
            )
            rhs = slot_total_demand[(y,s)]
            diff = lhs - rhs

            print("\n  Energy Balance Check:")
            print(f"    LHS = {lhs:.3f}, RHS = {rhs:.3f}, Difference = {diff:.6f}")

    print("\nAll reports generated successfully.")

else:
    print("Model not optimal. Status:", model.status)



 TOTAL COST OVER 5 YEARS 
Objective Value: 1,200,943.96


 YEAR 2025 RESULTS

---- CAPEX ----
Solar Bought This Year (kWp): 66.67
Battery Bought This Year (kWh): 91.67
50 kW Chargers Bought This Year: 0
400 kW Chargers Bought This Year: 1
Total CAPEX (USD): 60,000.00

---- INSTALLED CAPACITY (CUMULATIVE) ----
Total Battery Capacity Available (kWh): 91.67
Total 50 kW Chargers Available: 0
Total 400 kW Chargers Available: 1

---- OPEX ----
Grid OPEX per Year (USD): 23,998.07

---- EXPORT REVENUES ----

---- SLOT DETAILS ----

 Slot 1:
  Grid Energy Used (kWh): 438.32
  Solar:
    Generated: 0.00
    Used:      0.00
    Stored:    0.00
    Exported:  0.00
  Battery:
    SOC Start:  45.83
    Used:       45.83
    Exported:   0.00

  Energy Balance Check:
    LHS = 484.154, RHS = 484.154, Difference = 0.000000

 Slot 2:
  Grid Energy Used (kWh): 0.00
  Solar:
    Generated: 793.33
    Used:      705.98
    Stored:    0.00
    Exported:  87.35
  Battery:
    SOC Start:  0.00
    Used:     

## Exporting in report format , used AI (shown in appendix of report)

In [34]:
if model.status == GRB.OPTIMAL:
    
    results_data = {'Metric': []}
    for y in years:
        results_data[str(y)] = []
    
    def add_row(metric_name, year_values):
        results_data['Metric'].append(metric_name)
        for i, y in enumerate(years):
            results_data[str(y)].append(year_values[i])
    
    # Collect data
    capex_values = [capex_year[y].getValue() for y in years]
    opex_values = [opex_year[y].getValue() for y in years]
    charger50_values = [nocfifty[y].X for y in years]
    charger400_values = [nocfourhundred[y].X for y in years]
    solar_values = [solar_buy_year[y].X for y in years]
    battery_values = [battery_buy_year[y].X for y in years]
    
    # Total cost = CAPEX + OPEX (no revenue)
    total_cost_values = [capex_year[y].getValue() + opex_year[y].getValue() for y in years]
    
    # Add rows
    add_row('CAPEXCost', capex_values)
    add_row('OPEXCost', opex_values)
    add_row('#50KWChargers', charger50_values)
    add_row('#400KWChargers', charger400_values)
    add_row('SolarCapacity', solar_values)
    add_row('BatteryCapacity', battery_values)
    add_row('SolarExportRevenue', [0.0] * len(years))  # Always 0
    add_row('BatteryExportRevenue',[0.0]*len(years))
    add_row('TotalCost', total_cost_values)
    
    # Save to CSV
    df_results = pd.DataFrame(results_data)
    df_results.to_csv('optimization_results_no_export.csv', index=False)
    
    print("\n✓ Results saved to: optimization_results_no_export.csv")
    print(df_results)


✓ Results saved to: optimization_results_no_export.csv
                 Metric          2025           2026           2027  \
0             CAPEXCost  60000.000000   60000.000000   24090.586667   
1              OPEXCost  23998.067450   59731.713450  166533.405325   
2         #50KWChargers      0.000000       0.000000       0.000000   
3        #400KWChargers      1.000000       0.000000       0.000000   
4         SolarCapacity     66.666667     133.333333       0.000000   
5       BatteryCapacity     91.666667       0.000000      88.568333   
6    SolarExportRevenue      0.000000       0.000000       0.000000   
7  BatteryExportRevenue      0.000000       0.000000       0.000000   
8             TotalCost  83998.067450  119731.713450  190623.991992   

            2028           2029  
0    2500.000000       0.000000  
1  329661.030325  474429.155325  
2       0.000000       0.000000  
3       1.000000       0.000000  
4       0.000000       0.000000  
5       0.000000       0.0000

## Operational data, outputs csv file that has for each year, each slot how much solar energy was used,how much was exported, stored to the battery and so on

In [35]:
if model.status == GRB.OPTIMAL:
    
    operational_data = []
    
    for y in years:
        fleet_size = len(vehicle_df[vehicle_df['Year'] == y])
        
        for s in slots:
            row = {
                'Year': y,
                'Slot': s,
                'FleetSize': fleet_size,
                'ChargingRequirement_kWh': slot_charging_requirement[(y, s)],
                'BaseloadRequirement_kWh': slot_baseload[(y, s)],
                'TotalDemand_kWh': slot_total_demand[(y, s)],
                'SolarGenerated_kWh': solar_gen_year_slot[y, s].X,
                'SolarUsed_kWh': solar_used_year_slot[y, s].X,
                'SolarStored_kWh': solar_stored_battery_slot[y, s].X,
                'SolarExported_kWh': solar_export_year_slot[y, s].X,
                'BatterySOC_Start_kWh': battery_start_year_slot[y, s].X,
                'BatteryUsed_kWh': battery_used_year_slot[y, s].X,
                'GridUsed_kWh': grid_year_slot[y, s].X,
            }
            operational_data.append(row)
    
    df_operational = pd.DataFrame(operational_data)
    df_operational.to_csv('operational_details.csv', index=False)
    print("✓ Saved: operational_details.csv")
    print(df_operational.head(10))

✓ Saved: operational_details.csv
   Year  Slot  FleetSize  ChargingRequirement_kWh  BaseloadRequirement_kWh  \
0  2025     1         10                    477.0                   7.1542   
1  2025     2         10                    702.0                   3.9840   
2  2025     3         10                    729.0                   3.1175   
3  2025     4         10                    477.0                   7.0860   
4  2026     1         15                    477.0                   7.1542   
5  2026     2         15                   1197.0                   3.9840   
6  2026     3         15                   1233.0                   3.1175   
7  2026     4         15                    720.0                   7.0860   
8  2027     1         20                   1008.0                   7.1542   
9  2027     2         20                   1764.0                   3.9840   

   TotalDemand_kWh  SolarGenerated_kWh  SolarUsed_kWh  SolarStored_kWh  \
0         484.1542            0.00